In [1]:
import os
import pandas as pd
from anndata import read_h5ad
import scanpy as sc
import scregseg
import matplotlib.pyplot as plt

# Processing and preparing raw data

This tutorial shows how to create and manipulate count matrices.
Specifically, we shall illustrate:

    * How to create a count matrix using `scregseg fragments_to_counts` and `scregseg bam_to_counts`
    * How to filter and subset the count matrix using `scregseg filter` and `scregseg subset`
    * How to collapse cells within groups `scregseg collapse`
    * How to combine dataset using `scregseg merge`
    * How to create pseudobulk bam and bigwig tracks

### Obtaining the dataset

We will obtain the tutorial dataset from 10x Genomics.
Caution: The bam file is rather large. One might want to skip downloading it.

In [2]:
# first we download example scATAC-seq data
!wget -O atac_v1_pbmc_5k_fragments.tsv.gz https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz
!wget -O atac_v1_pbmc_5k_possorted_bam.bam https://cg.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_possorted_bam.bam
!wget -O atac_v1_pbmc_5k_possorted_bam.bam.bai https://cg.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_possorted_bam.bam.bai

# sometimes bedtools fails when processing the original *.tsv.gz files, but unpacking and packing seems to help
!gunzip -f atac_v1_pbmc_5k_fragments.tsv.gz
!gzip -f atac_v1_pbmc_5k_fragments.tsv

# prefiltered cells from CellRanger
!wget -O atac_v1_pbmc_5k_singlecell.csv https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_singlecell.csv

!wget -O atac_v1_pbmc_5k_analysis.tar.gz https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_analysis.tar.gz
!tar xvf atac_v1_pbmc_5k_analysis.tar.gz

--2021-02-20 00:51:13--  https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_fragments.tsv.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.0.173, 104.18.1.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.0.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 976353254 (931M) [text/tab-separated-values]
Saving to: ‘atac_v1_pbmc_5k_fragments.tsv.gz’

100%[======================================>] 976.353.254 77,1MB/s   in 12s    

2021-02-20 00:51:25 (79,5 MB/s) - ‘atac_v1_pbmc_5k_fragments.tsv.gz’ saved [976353254/976353254]

--2021-02-20 00:51:29--  https://cg.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_5k/atac_v1_pbmc_5k_possorted_bam.bam
Resolving cg.10xgenomics.com (cg.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cg.10xgenomics.com (cg.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response...

### Preparing a single-cell count matrix

First, be build genome-wide tiling windows. This will be used as the basis to construct the countmatrix.
The chromosome sizes are extracted from the fragments file. Alternatively, a bam-file could be used
for this step as well.

In [3]:
!scregseg make_tile \
          --regions tile1kb.bed \
          --binsize 1000 \
          --fragmentfile atac_v1_pbmc_5k_fragments.tsv.gz

Next, we construct a countmatrix. 
This step will require a fragments or bam-file (used with `scregseg bam_to_counts`)
and a bed-file specifying the genomic intervals.
The result of this step will be a regions by barcodes matrix.
The `--with-fraglen` determines that the fragment length information per interval is collected as well. This might be useful
for exploring informative states in the HMM-model later on.

In [4]:
!scregseg fragments_to_counts \
          --fragmentfile atac_v1_pbmc_5k_fragments.tsv.gz \
          --regions tile1kb.bed \
          --with-fraglen \
          --counts countmatrix.h5ad

... storing 'chrom' as categorical
... storing 'sample' as categorical


In the example, we'll save the countmatrix as AnnData dataset, which facilitates easy interoperability with scanpy.
Alternatively, one could save the countmatrix also as `countmatrix.mtx` to save the data in matrix market format.
The latter option makes it easier to continue with the dataset in a different environment, e.g. when using R.

Next, we subset/filter the raw countmatrix to remove poor quality barcodes.
The 10x Genomics data already contains information from the CellRanger pipeline about the cell quality.
So we can continue with the pre-determined high-quality cells.

To do so, we extract the desired cells (indicated by the is_cell_barcode column) from the 10x Genomics metadata:

In [5]:
df = pd.read_csv('atac_v1_pbmc_5k_singlecell.csv')
df = df[df.is__cell_barcode==1]
df[['barcode']].to_csv('qcontrolled_cells.csv', index=False)
print(f'{df.shape[0]} high-quality cells are left for downstream processing')

4654 high-quality cells are left for downstream processing


Then we subset the original countmatrix and retain only the quality controlled cells:

In [6]:
!scregseg subset \
          --incounts countmatrix.h5ad \
          --outcounts filtered_countmatrix.h5ad \
          --subset qcontrolled_cells.csv

In addition or as an alternative, it is possible to filter cells and regions using `scregseg filter`.

#!scregseg filter \
          --incounts countmatrix.h5ad \
          --outcounts filtered2_countmatrix.h5ad \
          --mincount 1000 \
          --maxcount 40000 \
          --trimcount 1

Now, let's check the content of the count matrix

adata = read_h5ad('filtered2_countmatrix.h5ad')
adata

In [7]:
adata = read_h5ad('filtered_countmatrix.h5ad')
adata

AnnData object with n_obs × n_vars = 2880143 × 4654
    obs: 'chrom', 'start', 'end'
    var: 'sample'
    obsm: 'frag_lens'

Sometimes it might be useful to concatenate count matrices, e.g. stemming from different experiments.
This can be achieved by using `scregseg merge`

In [8]:
!scregseg merge \
          --incounts filtered_countmatrix.h5ad filtered_countmatrix.h5ad \
          --outcounts merged_countmatrix.h5ad

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


### Preparing cell-group collapsed count matrices

After having performed some initial analysis, including feature identification,
dimensionality reduction and cell clustering, it might be of interest
to investigate the accessibility profiles across cell-groups or cell-clusters.
To this end, a count matrix can be constructed that collapses cell within groups: `scregseg collapse`

We shall utilize the pre-determined clustering results from CellRanger as an example to illustrate how to compile a pseudo-bulk count matrix.
In addition, we compile the pseudo-bulk count matrix with 500 bp resolution (for later use).

In [9]:
!scregseg make_tile \
          --regions tile500b.bed \
          --binsize 500 \
          --fragmentfile atac_v1_pbmc_5k_fragments.tsv.gz

In [10]:
!scregseg fragments_to_counts \
          --fragmentfile atac_v1_pbmc_5k_fragments.tsv.gz \
          --regions tile500b.bed \
          --counts countmatrix_500.h5ad

... storing 'chrom' as categorical
... storing 'sample' as categorical


In [11]:
!scregseg subset \
          --incounts countmatrix_500.h5ad \
          --outcounts filtered_countmatrix_500.h5ad \
          --subset qcontrolled_cells.csv

In [1]:
!scregseg filter \
          --incounts filtered_countmatrix_500.h5ad \
          --outcounts filtered_countmatrix_500.h5ad \
          --trimcount 1

Using TensorFlow backend.


In [2]:
!scregseg collapse \
          --incounts filtered_countmatrix_500.h5ad \
          --outcounts collapsed_countmatrix_500.h5ad \
          --cellgroup analysis/clustering/graphclust/clusters.csv

Using TensorFlow backend.
/home/wkopp/anaconda3/envs/scregseg2/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [21]:
psadata = read_h5ad('collapsed_countmatrix_500.h5ad')

In [22]:
psadata

AnnData object with n_obs × n_vars = 5760275 × 11
    obs: 'chrom', 'start', 'end'

The collapsed countmatrix contains 11 columns, each corresponding to one of the clusters
defined in the clusters.csv file.


Finally, if we have access to raw bam files, we could split the bamfiles into pseudobulk
tracks determined by `--cellgroup`. This will also generate associated bigwig files
if deeptools is installed/available.

In [18]:
!scregseg pseudobulk_tracks \
          --bamfile atac_v1_pbmc_5k_possorted_bam.bam \
          --barcodetag CB \
          --outdir pseudobulktracks \
          --cellgroup analysis/clustering/graphclust/clusters.csv

normalization: CPM
normalization: CPM
normalization: CPM
normalization: CPM
normalization: CPM
normalization: CPM
normalization: CPM
bamFilesList: ['pseudobulktracks/6.bam']
binLength: 50
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 1
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: False
chrsToSkip: []
stepSize: 50
center_read: False
samFlag_include: None
samFlag_exclude: None
minFragmentLength: 0
maxFragmentLength: 0
zerosToNans: False
smoothLength: None
save_data: False
out_file_for_raw_data: None
maxPairedFragmentLength: 1000
bamFilesList: ['pseudobulktracks/7.bam']
binLength: 50
numberOfSamples: None
blackListFileName: None
skipZeroOverZero: False
bed_and_bin: False
genomeChunkSize: None
defaultFragmentLength: read length
numberOfProcessors: 1
verbose: False
region: None
bedFile: None
minMappingQuality: None
ignoreDuplicates: F

In [19]:
os.listdir('pseudobulktracks')

['1.bam',
 '2.bam',
 '3.bam',
 '4.bam',
 '5.bam',
 '6.bam',
 '7.bam',
 '8.bam',
 '9.bam',
 '10.bam',
 '11.bam',
 '1.bam.bai',
 '2.bam.bai',
 '3.bam.bai',
 '4.bam.bai',
 '5.bam.bai',
 '6.bam.bai',
 '7.bam.bai',
 '8.bam.bai',
 '9.bam.bai',
 '10.bam.bai',
 '11.bam.bai',
 '7.bigwig',
 '8.bigwig',
 '9.bigwig',
 '10.bigwig',
 '3.bigwig',
 '4.bigwig',
 '2.bigwig',
 '6.bigwig',
 '11.bigwig',
 '1.bigwig',
 '5.bigwig']